In [6]:
import pandas as pd

In [ ]:
import base64
import pandas as pd
import requests as rq
import json
from datetime import date

# ================
# 1. TOKEN
# ================
def get_outh_token():
    Apikey = '0gs9fabdhesbisczsyx2pjhvvxq8hg9z'
    Secret = 'SoWh9TefKizy'

    message = f"{Apikey}:{Secret}"
    auth = "Basic " + base64.b64encode(message.encode("ascii")).decode("ascii")

    headers_dic = {
        "Authorization": auth,
        "Content-Type": "application/x-www-form-urlencoded;charset=UTF-8"
    }
    data_dic = {
        "grant_type": "client_credentials",
        "scope": "read"
    }

    r = rq.post("https://api.idealista.com/oauth/token", headers=headers_dic, data=data_dic)
    if r.status_code != 200:
        raise Exception(f"Error al obtener token: {r.status_code} - {r.text}")

    return r.json()['access_token']

# ================
# 2. BÚSQUEDA (POR PÁGINA)
# ================
def search_api(pagination=1):
    token = get_outh_token()
    headers = {'Authorization': 'Bearer ' + token}

    data = {
        "operation": "rent",
        "propertyType": "homes",
        "center": "37.365733231921354,-6.007946780316465",
        "distance": 7000,  # en metros (7 km)
        "maxItems": 50,
        "numPage": pagination,
        "language": "es",
        "country": "es"
    }

    r = rq.post("https://api.idealista.com/3.5/es/search", headers=headers, data=data)

    if r.status_code != 200:
        raise Exception(f"Error en búsqueda: {r.status_code} - {r.text}")

    return r.json()

# ================
# 3. RECORRER HASTA 3000 CASAS
# ================
all_houses = []
total_pages = 45  # 3000 casas = 60 páginas de 50 resultados

for page in range(1, total_pages + 1):
    print(f"Descargando página {page}/{total_pages}...")
    data = search_api(page)

    houses = data.get("elementList", [])
    all_houses.extend(houses)

    # Si ya no hay más resultados, paramos
    if not houses:
        break

print(f"✅ Total casas descargadas: {len(all_houses)}")

# ================
# 4. CONVERTIR A DATAFRAME
# ================
df = pd.DataFrame(all_houses)

columnas_interes = [
    "propertyCode", "price", "propertyType", "operation",
    "rooms", "bathrooms", "size", "address", "province",
    "municipality", "district", "neighborhood", "latitude",
    "longitude", "distance", "exterior", "floor", "parkingSpace", "priceByArea", "detailedType"
]
df = df[columnas_interes]

# ================
# 5. GUARDAR EN CSV
# ================
nombre_csv = f"Alquiler_sevilla_{date.today()}.csv"
df.to_csv(nombre_csv, index=False, encoding="utf-8-sig")

print(f"📂 CSV guardado como: {nombre_csv}")

Descargando página 1/45...
Descargando página 2/45...
Descargando página 3/45...
Descargando página 4/45...
Descargando página 5/45...
Descargando página 6/45...
Descargando página 7/45...
Descargando página 8/45...
Descargando página 9/45...
Descargando página 10/45...
Descargando página 11/45...
Descargando página 12/45...
Descargando página 13/45...
Descargando página 14/45...
Descargando página 15/45...
Descargando página 16/45...
Descargando página 17/45...
Descargando página 18/45...
Descargando página 19/45...
Descargando página 20/45...
Descargando página 21/45...
Descargando página 22/45...
Descargando página 23/45...
Descargando página 24/45...
Descargando página 25/45...
Descargando página 26/45...
✅ Total casas descargadas: 1230
📂 CSV guardado como: ventas_sevilla_2025-07-23.csv


In [7]:
ruta = 'Alquiler_sevilla_2025-07-23.csv'
df = pd.read_csv(ruta)

In [4]:
df.columns

Index(['propertyCode', 'price', 'propertyType', 'operation', 'rooms',
       'bathrooms', 'size', 'address', 'province', 'municipality', 'district',
       'neighborhood', 'latitude', 'longitude', 'distance', 'exterior',
       'floor', 'parkingSpace', 'priceByArea', 'detailedType'],
      dtype='object')

In [8]:
df['district'].unique()

array(['Prado de San Sebastián - Felipe II - Bueno Monreal', 'Nervión',
       'La Palmera - Los Bermejales', 'Macarena', 'Centro',
       'Los Remedios', 'Cerro Amate',
       'Santa Justa - Miraflores - Cruz Roja', 'Condequinto',
       'Ciudad Aljarafe', 'Las Almenas', nan, 'Santa Clara', 'San Pablo',
       'Triana', 'Bellavista - Jardines de Hércules', 'Lepanto',
       'Zona Universitaria', 'Simon Verde', 'Barrio Alto', 'Barrio Bajo',
       'Pino Montano', 'Aljamar', 'Santa Eufemia',
       'El Zaudín - Club de Golf', 'Ciudad Expo', 'Sevilla este',
       'Zona Avda. Juan Diego', 'Arco norte - Avda España',
       'San Jerónimo', 'Nuevo Bulevar', 'Zona Avenida de Europa'],
      dtype=object)

In [5]:
df['parkingSpace'].unique()

array(["{'hasParkingSpace': True, 'isParkingSpaceIncludedInPrice': True}",
       nan,
       "{'hasParkingSpace': True, 'isParkingSpaceIncludedInPrice': False, 'parkingSpacePrice': 35.0}",
       "{'hasParkingSpace': True, 'isParkingSpaceIncludedInPrice': False, 'parkingSpacePrice': 40.0}",
       "{'hasParkingSpace': True, 'isParkingSpaceIncludedInPrice': False, 'parkingSpacePrice': 100.0}",
       "{'hasParkingSpace': True, 'isParkingSpaceIncludedInPrice': False, 'parkingSpacePrice': 140.0}",
       "{'hasParkingSpace': True, 'isParkingSpaceIncludedInPrice': False, 'parkingSpacePrice': 150.0}",
       "{'hasParkingSpace': True, 'isParkingSpaceIncludedInPrice': False, 'parkingSpacePrice': 50.0}",
       "{'hasParkingSpace': True, 'isParkingSpaceIncludedInPrice': False, 'parkingSpacePrice': 190.0}",
       "{'hasParkingSpace': True, 'isParkingSpaceIncludedInPrice': False, 'parkingSpacePrice': 75.0}",
       "{'hasParkingSpace': True, 'isParkingSpaceIncludedInPrice': False, 'parkingSpa

In [3]:
df["bathrooms"].value_counts()

bathrooms
1    790
2    363
3     54
4     16
5      4
6      2
8      1
Name: count, dtype: int64

In [4]:
df["rooms"].value_counts()

rooms
3    438
2    295
1    274
4    133
0     61
5     25
7      2
6      2
Name: count, dtype: int64

In [5]:
df["floor"].value_counts()

floor
1     296
2     283
3     164
bj    162
4     101
5      71
6      28
en     27
7      19
8       8
9       5
10      4
11      4
12      3
ss      2
Name: count, dtype: int64